In [14]:
import pandas as pd
import playlists
import clustering
import dataset_utils
import joblib

from difflib import SequenceMatcher

In [15]:
playlists_dataset = playlists.load_dataset()

In [16]:
# rename columns - user_id	"artistname"	"trackname"	"playlistname"
playlists_dataset.rename(
    columns={
        "user_id": "user_id",
        ' "artistname"': "artist",
        ' "trackname"': "track",
        ' "playlistname"': "playlistname",
    },
    inplace=True,
)

In [17]:
playlists_dataset.columns

Index(['user_id', 'artist', 'track', 'playlistname'], dtype='object')

In [18]:

song_features = clustering.load_dataset()
df_tract_details = dataset_utils.load_track_details()

df_artist_release_keys = dataset_utils.load_artist_release()
df_artist_details = dataset_utils.load_artist_details()

/Users/tzarski/Documents/Development/cs_studies/wut-earin-laboratory/project/dataset_utils.py:16: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/10-m-tracks/bp_track.csv")


In [19]:
# remove duplicates columns after merge
song_features = pd.merge(song_features, df_tract_details, on="isrc", how="inner")

song_features = song_features.drop_duplicates(subset=["isrc"])

df_release_artist_names = pd.merge(
    df_artist_release_keys, df_artist_details, on="artist_id", how="inner"
)


song_features = pd.merge(
    song_features, df_release_artist_names, on="release_id", how="inner"
)

song_features.info()

del df_tract_details
del df_artist_release_keys
del df_artist_details
del df_release_artist_names

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46824672 entries, 0 to 46824671
Data columns (total 36 columns):
 #   Column            Dtype  
---  ------            -----  
 0   isrc              object 
 1   acousticness      float64
 2   danceability      float64
 3   duration_ms_x     int64  
 4   energy            float64
 5   instrumentalness  float64
 6   key               int64  
 7   liveness          float64
 8   loudness          float64
 9   mode              int64  
 10  speechiness       float64
 11  tempo             int64  
 12  time_signature    int64  
 13  valence           float64
 14  updated_on_x_x    object 
 15  track_id          int64  
 16  title             object 
 17  mix               object 
 18  is_remixed        object 
 19  release_date      object 
 20  genre_id          int64  
 21  subgenre_id       float64
 22  track_url         object 
 23  bpm               int64  
 24  duration          object 
 25  duration_ms_y     float64
 26  key_id      

In [20]:
unique_song_names = playlists_dataset["track"].unique()

dataset_playlist_traks = song_features[song_features["title"].isin(unique_song_names)]



In [21]:
unique_song_names = dataset_playlist_traks["title"].unique()
print("unique_song_names", len(unique_song_names))

# add is as a hash of the track name
dataset_playlist_traks["track_id"] = dataset_playlist_traks["title"].apply(
    lambda x: hash(x)
)

# filter out songs that are not in unique_song_names

dataset_playlist_traks = dataset_playlist_traks[
    dataset_playlist_traks["title"].isin(unique_song_names)
]



unique_song_names 238117


/var/folders/90/czs0nv0d30x0yxcfw1f2b1m00000gn/T/ipykernel_45344/3598854012.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_playlist_traks["track_id"] = dataset_playlist_traks["title"].apply(


In [22]:
# reorder playlist dataset, remove user_id column, reformat dataset, take only unique songs and create playlists columns that will contain all sthe playlists that contain the song
playlist_dataset_refined = playlists_dataset.drop(columns=["user_id"])

# aggregate the songs but keep the playlists names in a list
playlist_dataset_refined = playlist_dataset_refined.groupby(["artist", "track"]).agg(
    {"playlistname": list}
)

playlist_dataset_refined.reset_index(inplace=True)

playlist_dataset_refined["playlistname"] = playlist_dataset_refined["playlistname"].apply(
    lambda x: list(set(x))
)

# make an id of hash of the track name
playlist_dataset_refined["track_id"] = playlist_dataset_refined["track"].apply(
    lambda x: hash(x)
)

# merge the two datasets




In [23]:
del song_features
del playlists_dataset

playlist_dataset_refined.drop_duplicates(subset=["track_id"], inplace=True)
dataset_playlist_traks.drop_duplicates(subset=["track_id"], inplace=True)



In [26]:
dataset_playlist_tracks = pd.merge(
    playlist_dataset_refined, dataset_playlist_traks, on="track_id", how="left"
)


dataset_playlist_tracks_columns = dataset_playlist_tracks.columns
feature_columns = [
    "artist",
    "track",
    "playlistname",
    "isrc",
    "acousticness",
    "danceability",
    "energy",
    "instrumentalness",
    "key",
    "liveness",
    "loudness",
    "mode",
    "speechiness",
    "tempo",
    "valence",
    "title",
    "mix",
    "is_remixed",
    "release_date",
    "genre_id",
    "subgenre_id",
    "track_url",
    "bpm",
    "duration",
    "key_id",
    "release_id",
    "artist_name",
]


final_dataset = dataset_playlist_tracks[feature_columns]

In [27]:
joblib.dump(final_dataset, "playlist_dataset")

Index(['artist', 'track', 'playlistname', 'isrc', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'valence', 'title', 'mix',
       'is_remixed', 'release_date', 'genre_id', 'subgenre_id', 'track_url',
       'bpm', 'duration', 'key_id', 'release_id', 'artist_name'],
      dtype='object')